In [35]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os.path
import glob
import os

همه = 0
سیاسی = 1
اخبار سلامت = 39
واژه خانه = 43
که بوذ و چه کرد = 44
قرآن = 45
اقتصادی = 4
اجتماعی = 5
ورزشی = 6
سلامت = 25
بین الملل = 3
سیاست خارجی = 2
فرهنگی هنری = 8
محیط زیست = 29
فناوری = 14
علمی = 7
حوادث = 9
سرگرمی = 15
عمومی = 21
کاربران = 22
کافه گفت و گو = 23
سفر و تفریح = 24

In [52]:
def extract_news(start_date = '1401/01/01',end_date = '1401/01/28',subject_id = 0):
    page_number = 1
    url = 'https://www.asriran.com/fa/archive?service_id=1&sec_id=-1&cat_id='+str(subject_id)+'&rpp=100&from_date='+start_date+'&to_date='+end_date+'&p='
    while True:
        # check if the current page's data exists
        if (os.path.isfile('.\\data'+str(page_number)+'.csv')):
            print('there is already page '+str(page_number))
            page_number += 1
            continue

        upperframe=[] 
        url1 = url + str(page_number)
        page = requests.get(url1)
        soup = BeautifulSoup(page.text, "html.parser")

        articles=soup.find_all('article',attrs={'vizhe_cv col-xs-12 col-sm-6'})
        if len(articles)==0 :
            print('finished')
            break
        print('page '+ str(page_number))

        for article in articles:
            title = article.find("a",attrs={'class':"vizhe_title"}).text.strip()
            link = article.find("a").get('href').strip()
            link = 'https://www.asriran.com' + link

            summary = article.find("div",attrs={'class':"vizhe_lead"}).text.strip()
            date = article.find("span",attrs={'class':"tarikh_archive"}).text.strip()

            url2 = link
            page2 = requests.get(url2)
            soup2 = BeautifulSoup(page2.text, "html.parser")

            # IF THE PAGE IS GONE
            ERROR = soup2.find("div",attrs={'class':"error_container"})
            if ERROR :
                subject = ''
                body = 'صفحه درخواستی شما موجود نمی باشد'
                news_id = ''
                short_link = ''
                frame = [news_id,date,subject,title,summary,body,short_link]
                upperframe.append(frame)
                continue # jump to the next article


            subject = soup2.find("div",attrs={'class':"news_path"}).find_all("a")[-1].text

            body = soup2.find("div",attrs={'class':"body"}).text.strip()[:-155]

            news_id = soup2.find("div",attrs={'class':"news_nav news_id_c"}).text.strip()
            news_id = int(re.findall(r'\d+', news_id)[0])

            short_link = soup2.find("div",attrs={'class':"short-link row"}).find("a").get('href').strip()
            short_link = 'https://www.asriran.com' + short_link

            
            frame = [news_id,date,subject,title,summary,body,short_link]
            upperframe.append(frame)

        data=pd.DataFrame(upperframe, columns=['News_ID','Date','Subject','Title','Summary','Body','Short_link'])
        data.to_csv('data'+str(page_number)+'.csv', encoding='utf-8', index=False)
        page_number += 1

    #data=pd.DataFrame(upperframe, columns=['News_ID','Date','Subject','Title','Summary','Body','Short_link'])

    return print("DONE")

extract_news(start_date = '1401/01/01',end_date = '1401/01/28',subject_id = 0)

there is already page 1
there is already page 2
there is already page 3
there is already page 4
there is already page 5
there is already page 6
there is already page 7
there is already page 8
there is already page 9
there is already page 10
there is already page 11
there is already page 12
there is already page 13
there is already page 14
there is already page 15
there is already page 16
there is already page 17
there is already page 18
there is already page 19
there is already page 20
there is already page 21
there is already page 22
there is already page 23
there is already page 24
there is already page 25
page 26
there is already page 27
there is already page 28
there is already page 29
there is already page 30
there is already page 31
there is already page 32
there is already page 33
there is already page 34
there is already page 35
there is already page 36
there is already page 37
there is already page 38
finished
DONE


In [55]:
# setting the path for joining multiple files
files = os.path.join(".\\", "data*.csv")

# list of merged files returned
files = glob.glob(files)

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)

# convert df to csv
df.to_csv('data.csv', encoding='utf-8', index=False)